<a href="https://colab.research.google.com/github/WittmannF/deep-learning-keras/blob/master/modulo-2/atividade-1-boston-housing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Tarefa 1: Prevendo Preços das Casas de Boston
Em sua primeira missão, você vai treinar um modelo com um conjunto de dados real. O [Boston Housing Dataset](http://lib.stat.cmu.edu/datasets/boston) contém informações recolhidas em 1978 pelo U.S Census Serviço com relação a habitação na área de Boston Mass. Seu objetivo é criar um modelo de previsão do preço de uma casa com base em 13 atributos (características). Essas características são:
- CRIM: Esta é a taxa de criminalidade per capita por cidade
- ZN: Esta é a proporção de terrenos residenciais zoneada para lotes maiores do que 25.000 sq.ft.
- INDUS: Proporção de não retalhistas acres de negócios por cidade
- CHAS: Charles River variável binária (isto é igual a 1 se limita tracto rio; 0 de outro modo)
- NOX: nítrico concentração de óxidos de (partes por 10 milhões)
- RM: Número médio de quartos por habitação
- AGE: Proporção de unidades ocupadas pelos proprietários construído antes de 1940
- DIS: distâncias ponderados para ve centros de emprego Boston
- RAD: Índice de acessibilidade às rodovias radiais
- TAX: taxa de propriedade de impostos Full-valor por 10000 USD
- PTRATIO: relação professor-aluno por cidade
- B: Calculado como 1000 (Bk - 0.63) ^ 2, onde Bk é a proporção de pessoas de ascendência Africano americano por cidade
- LSTAT: Percentagem da população status inferior

Como variável alvo, vamos usar última coluna:
- MEDV: valor médio das casas ocupadas pelos proprietários

Opcionalmente, você também pode comparar seus resultados com o [Boston Housing Data Science Contest by Kaggle](https://www.kaggle.com/c/boston-housing). Embora o concurso já terminou, você ainda pode fazer uma apresentação tardia e comparar sua pontuação com o leaderboard.

## Começando
Você terá que percorrer os 5 passos que temos visto no vídeo 4:
1. Explorando os dados
   - Importação de dados
   - Compreender os dados
2. Preparar os dados
   - Scaling
   - Transformando
   - One-Hot Encoding
   - Train teste de divisão /
3. Desenvolver um modelo base
4. Verificação de Previsões
5. Melhoria de Resultados
6. (Opcional) Compare seus resultados no Kaggle
Os 4 primeiros passos são parcialmente implementado. Você terá que acabar com eles e também implementar a etapa 5.
## 1. Explorando os dados
### Importando o conjunto de dados
O conjunto de dados Boston Housing está disponível em [keras.io/datasets](https://keras.io/datasets/). Executar a próxima célula, a fim de fazer o download e importar o conjunto de dados:

In [0]:
import warnings
warnings.filterwarnings("ignore")

from keras.datasets import boston_housing

(X_train, y_train), (X_test, y_test) = boston_housing.load_data()

Vamos explorar e compreender este conjunto de dados.
### Compreender o conjunto de dados (Análise Exploratória de Dados)
O conjunto de dados já foi splited em ( `X_train` e` y_train`) e teste ( `X_test` e` y_test`) subconjuntos de treinamento. Como uma revisão, o conjunto de treinamento é usado para definir a fronteira de decisão da rede neural (ou seja, treinamento do modelo), enquanto o conjunto de teste é um conjunto independente usados ​​para avaliar o quão bom ele é o modelo com dados invisíveis. Vamos obter algumas informações a partir desses conjuntos de dados. Vamos converter o conjunto de treinamento em um [Pandas Dataframe](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.html) a fim de obter mais estatísticas:

In [0]:
import pandas as pd
import numpy as np

# TODO: Number of elements in the training set
training_len = len(y_train)

# TODO: Number of elements in the test set
test_len = len(y_test)

# Show the calculated values
print("There are {} houses in the training set".format(training_len))
print("There are {} houses in the test set".format(test_len))

# Convert the training set into a Pandas Dataframe in order to get more statistics:
columns = ['CRIM', 'ZN', 'INDUS', 'CHAS', 'NOX', 'RM', 'AGE', 'DIS', 'RAD', 'TAX', 'PTRATIO', 'B', 'LSTAT', 'PRICE']
df = pd.DataFrame(np.c_[X_train, y_train], columns = columns)
df.head()

A última coluna foi renomeado de ** MEDV ** Para Preço ** **, a fim de fornecer mais semântica sobre o que queremos de prever. Agora, vamos obter algumas estatísticas sobre cada recurso:

In [0]:
df.describe()

E, finalmente, vamos visualizar algumas dessas colunas com uma matriz de dispersão usando [seaborn](https://seaborn.pydata.org/):

In [0]:
import seaborn as sns # pip install seaborn
cols = ['LSTAT', 'INDUS', 'NOX', 'RM', 'PRICE'] 
sns.pairplot(df[cols], kind='reg', plot_kws={'line_kws':{'color':'orange'}})
# NOTE: If there's no output, run again!

No gráfico anterior, podemos verificar as tendências de alguns recursos quando comparado com os outros. Especialmente a última linha é importante, pois permite-nos verificar as tendências de algumas características com o preço no eixo y. Também é possível verificar algumas parcelas do histograma.
> ** NOTA: ** Se a linha anterior não funcionar, é provável porque Seaborn não está instalado. Se você estiver usando o chapéu seu caso, apenas `pip instalar seaborn` em seu terminal. Se você estiver usando o Google Colab, o Seaborn já está instalado por padrão.
## 2. Preparar os dados
Uma vez que o conjunto de dados já está dividida em treinamento e subconjuntos de testes, só está faltando para dimensionar o conjunto de dados.
### 2.1 Atribuição - Escala os dados de treinamento
Dimensionar o conjunto de dados usando scaller padrão, da mesma forma como foi apresentado no vídeo 4:

In [0]:
print("*** Before Scaling ***")
print("Mean of each feature in the training set\n{}".format(np.round(np.mean(X_train, axis=0))))
print("Standard deviation of each feature in the training set\n{}".format(np.round(np.std(X_train, axis=0))))

# Function to standardize the training data
def standardize(X, X_mean, X_std):
    return (X-X_mean)/X_std

# Get mean of each column in the training set
X_mean = X_train.mean(axis=0)
X_std = X_train.std(axis=0)

# Apply standardization to the features in the training and test sets
X_train = standardize(X_train, X_mean, X_std)
X_test = standardize(X_test, X_mean, X_std)

print("\n*** After Scaling ***")
print("Mean of each feature in the training set\n{}".format(np.round(np.mean(X_train, axis=0))))
print("Standard deviation of each feature in the training set\n{}".format(np.round(np.std(X_train, axis=0))))

## 3. Desenvolver um modelo básico
Para a função de perda, vamos usar o mesmo que é exigido no concurso Kaggle, o erro Root Mean Squared (RMSE):
$$ RMSE = \sqrt {\sum_i^N {\frac{(y_i - \hat{y_i})^2}{N}}} $$


In [0]:
from keras import backend as K

def root_mean_squared_error(y_true, y_pred):
    return K.sqrt(K.mean(K.square(y_pred - y_true), axis=-1)) 


Agora vamos executar o primeiro e mais simples versão do nosso modelo, da mesma forma como foi feito em vídeo 4:

In [0]:
# 0. Import keras dependencies here
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import SGD

# 1. Define your base model here
model = Sequential([
        Dense(units=1, input_shape=(13,))
    ])

# 2. Set your optimizer and loss function here
opt = SGD()
model.compile(optimizer=opt,
             loss=[root_mean_squared_error])


# 3. Train your model
model.fit(X_train, y_train)

Quando comparado com o Video 4, a forma de entrada foi aumentada para 13 uma vez que existem 13 características.

## 4. Previsões Verificação
Vamos verificar alguns resultados numéricos e visuais:

In [0]:
import matplotlib.pyplot as plt
%matplotlib inline

def check_predictions(model, X, y, cols):
    print("\n**** Evaluating the Test set ****")
    loss_test = model.evaluate(X, y)
    print("Test Loss: {:.3f}".format(loss_test))
    
    for column in cols:
        y_pred = model.predict(X)
        idx = columns.index(column)
        plt.scatter(X[:, idx], y, c='b', alpha=0.5, label="True Data")
        plt.scatter(X[:, idx], y_pred, c='orange', alpha=0.5, label="Predictions")
        plt.xlabel(column)
        plt.ylabel("Price")
        plt.legend(loc=0)
        plt.show()
    
check_predictions(model, X_test, y_test, ['RM', 'LSTAT'])

Nós podemos verificar tanto numericamente e visualmente que os resultados são muito ruins. Idealmente queremos um R2 Score próximo de 1. O valor de um dos -7 indica que o modelo é muito ruim. Nós um também confirmam isso quando comparando os verdadeiros dados e previsões em duas características. Seu trabalho vai ser para melhorar esses resultados:
## 5. TODO: Melhorar os Resultados

Agora é a sua vez! Tente-se algumas técnicas para melhorar os resultados anteriores. Aqui estão algumas ideias:
- Aumentar o número de camadas ocultas e funções de ativação add:
- Como [regra de ouro](https://stats.stackexchange.com/questions/181/how-to-choose-the-number-of-hidden-layers-and-nodes-in-a-feedforward-neural-netw), você pode começar a tentar uma camada escondida com o número de neurônios nessa camada como a média dos neurônios nas camadas de entrada, mais saída.
- Este é um exemplo com **duas camadas escondidas** com 13 neuronios na primeira camada escondida e 7 neuronios na segunda camada escondida:
    ```
    model = Sequential([
    Dense(units=13, input_shape=(13,), activation='relu'),
    Dense(units=7, activation='relu'),
    Dense(1)
    ])
    ```

- Aumentar o número de épocas
- Tune a taxa de aprendizagem
- Alterar o otimizador


In [0]:
# 0. TODO: Import keras dependencies here
from keras.models import ...
from keras.layers import ...
from keras.optimizers import ...

# 1. TODO: Define your base model here
model = Sequential([
        ...
    ])


# 2. TODO: Set your optimizer and loss function here
opt = ...(lr=...)

model.compile(optimizer=opt,
             loss=[root_mean_squared_error])

# 3. Train your model
model.fit(X_train, y_train, epochs=...)

Agora vamos ver a previsão do conjunto de teste:

In [0]:
check_predictions(model, X_test, y_test, ['RM', 'LSTAT'])

## 6. (Opcional) Enviar para Kaggle
Embora o concurso já está concluído, você pode tentar enviar como uma apresentação tardia em Kaggle e comparar sua pontuação com outros concorrentes no [leaderboard](https://www.kaggle.com/c/boston-housing/leaderboard). Você apenas tem de clicar no botão "Late Submission" no [contest page](https://www.kaggle.com/c/boston-housing) e, em seguida, fazer upload de um arquivo CSV no formato de apresentação do concurso (ID e previsões).
A fim de tentar-se, certifique-se primeiro o download do arquivo test.csv em [kaggle.com/c/boston-housing/data](https://www.kaggle.com/c/boston-housing/data) e adicioná-lo na pasta do seu projeto. Se você estiver usando Colab, você pode ter que montar Google Drive pela primeira vez.

In [0]:
USING_COLAB = False

if USING_COLAB:
    from google.colab import drive
    drive.mount('/content/drive')
    # Update here with the folder of the files of your course
    !cd '/content/drive/My Drive/course/day-1'
    

TEST_PATH = 'kaggle_boston_housing_test.csv'

# Load the test.csv file
testdf = pd.read_csv(TEST_PATH)
testdf.head()

Em seguida, execute o seguinte célula, a fim de criar um arquivo de submissão CSV com base em seu modelo:

In [0]:
# Get the column with the IDs
ids = testdf['ID'].values

# Apply Standard transform in the test set
X_test_kaggle = testdf.drop('ID', axis=1)
X_test_kaggle_std = sc.fit_transform(X_test_kaggle)

# Get predictions
y_pred_kaggle = model.predict(X_test_kaggle_std)

# Convert to a dataframe in the submission format
submit = pd.DataFrame(np.c_[ids, y_pred_kaggle], columns=['ID', 'medv'])
submit['ID'] = submit['ID'].astype(int)

# Convert to a CSV file
submit.to_csv('my_submission_improved_model.csv', index=False)

Abaixo está a minha melhor pontuação. Você pode comparar seu melhor pontuação usando o [leaderboard](https://www.kaggle.com/c/boston-housing/leaderboard):
![screen shot 2019-02-15 at 18 14 04](https://user-images.githubusercontent.com/5733246/52882019-824f7900-314d-11e9-97d7-17e48e4a1770.png)
